#### 1. **프로젝트 개요**

* 분석 목적: 일본 애니메이션 제작사 인기 순위를 만들고, 그것을 활용하여 사용자들이 궁금해 할 만한 정보들을 시각화 하기 위함
* 데이터 출처 및 설명: anilist graphql API 호출로 얻어낸 데이터들을 DataFrame 형식으로 저장 후 가공해 사용


#### 2. **데이터 기본 정보 확인**

* 행/열 수 2500/7
* 각 변수 이름, 타입, 결측치 여부 등
id(int)		seasonYear(float64)	popularity(int64)	favourites(int64)	genres(object)    studio(object)	PopularitySum(int64)
```markdown
## 2. 데이터 기본 정보
- 총 데이터 수: 2500개
- 컬럼 수: 7개
- 주요 컬럼: `popularity`, `genres`, `studio` 등
```



#### 3. **기술 통계 및 데이터 요약**

* 수치형 변수: 평균, 중앙값, 표준편차
* 범주형 변수: 각 범주의 빈도수, 빈도수의 평균, 표준편차. 최소/최대 등

```markdown
## 3. 기술 통계 요약

### 수치형 변수

| 컬럼명           | 평균    | 중앙값  | 표준편차 | 최솟값 | 최댓값 |
|------------------|--------|--------|---------|--------|--------|
| Popularity   | 89011.7452  | 51950.5  | 101006.06   | 21759  | 923138 |
| favourites   | 2369.6564  | 809  | 5116   | 63  | 89649 |
### 범주형 변수

| 컬럼명    | 고유값 | 최빈값 | 분포                           |
|-----------|--------|--------|--------------------------------|
| title    | 2500개    | x   | x       |
| genres     | 18    |   Comedy   | x       |
```




## 4. 결측치 및 이상치 탐색
- 결측치 없음, 이상치 모두 활용(전처리 필요 없음)




#### 5. **변수 간 관계 분석**

* 시각화: scatter, regplot, plot
* 상관분석: 수치형 변수 간 상관계수, 범주형 변수의 빈도수 등
* 그룹 비교: 상위 1~10위권과 11~20위권의 비교 등


## 5. 변수 간 관계 분석
- `PopularitySum`와 `favourities` 관계: 인기가 높은 제작사의 작품일 수록 좋아요 수가 많다.
- `favourities`와 `polularity` 관계: 좋아요가 많은 작품일 수록 인기가 많다.

## 6. 파생 변수 및 전처리
- 상위 2500개의 애니메이션을 만든 `studio` 중 `isMain: true` 인 제작사에 그 작품의 `“popularity”` 값을 합한 순위
- `Studio` → 구간화


## 7. 요약 및 인사이트
- Notion-제작사 인기 순위 활용 참조
https://morning-sesame-d7e.notion.site/3-2a945ffc66ac800e8415ecdea1ee6afb?pvs=74

- 과연 좋아요를 많이 받은 작품일수록 유명한 제작사가 그렸을까?
    
    scatter :
    
    x: 제작사 인기 순위 (오른쪽이 유명), y: 그 제작사가 만든 작품의 `“favourities”` 개수
    
    ![](https://morning-sesame-d7e.notion.site/image/attachment%3A055330be-c9a6-46c7-962b-9478dbfcb24f%3Aimage.png?table=block&id=2b845ffc-66ac-8006-8fe0-f3d206512102&spaceId=9bd45ffc-66ac-8182-98f0-000382468b0b&width=770&userId=&cache=v2)

    
- 유명한 제작사일수록 만드는 작품들의 장르가 다양할까?
    
    scatter, regplot(sns)
    
    - x축: 제작사 인기 순위
    - y축: 해당 제작사의 유니크 장르 수) `"genre_count”`
    - **`ci=95`** 회귀선(단순 선형 회귀)의 **신뢰구간(confidence interval)** 폭을 95%로 계산
    - 파란 띠가 “회귀 추정의 불확실성”이고, 데이터 퍼짐이 커질수록 밴드도 넓어지는 경향을 보게 된다

    ![](https://morning-sesame-d7e.notion.site/image/attachment%3A01cb3b67-dead-4591-9194-55a87bd0685d%3Aimage.png?table=block&id=2b845ffc-66ac-80a1-933c-f9d4c1943729&spaceId=9bd45ffc-66ac-8182-98f0-000382468b0b&width=960&userId=&cache=v2)

- 유명한 제작사일수록 오랫동안 활동했을까?
    - 계산:
        - 각 제작사에 대해:
            - 이 제작사가 포함된 모든 작품의 **`seasonYear`** 목록을 모으고
            - **`max(seasonYear) - min(seasonYear)`** = 활동 기간(년 단위)
    - 출력:
        - x: 스튜디오 인기 순위 1~100 (오른쪽일수록 인기 많음)
        - y: `groupby`,**`.agg(mean_span="mean", max_span="max")`** 각 구간의 평균, 최대값을 구한다

    ![](https://morning-sesame-d7e.notion.site/image/attachment%3A20d50522-ee7a-467f-96ac-626b349d7a40%3Aimage.png?table=block&id=2b845ffc-66ac-806f-b0d1-cdc87f68581b&spaceId=9bd45ffc-66ac-8182-98f0-000382468b0b&width=860&userId=&cache=v2)